In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from modules._import_helper import safe_import_from
import time

# Import filters
set_seed = safe_import_from('00_repo_standards.src.mlphys_core', 'set_seed')
(ExtendedKalmanFilter, pendulum_dynamics, angle_observation_model) = safe_import_from(
    '04_time_series_state_space.src.ekf',
    'ExtendedKalmanFilter', 'pendulum_dynamics', 'angle_observation_model'
)
(ParticleFilter, gaussian_likelihood, create_process_noise_wrapper) = safe_import_from(
    '04_time_series_state_space.src.particle_filter',
    'ParticleFilter', 'gaussian_likelihood', 'create_process_noise_wrapper'
)

set_seed(42)
plt.style.use('default')

output_dir = Path('modules/04_time_series_state_space/reports/nb03_nonlinear_estimation')
output_dir.mkdir(parents=True, exist_ok=True)

print("✓ Setup complete")

## 4. Nonlinear System: Pendulum

Dynamics: 
$$
\begin{aligned}
\theta_{k+1} &= \theta_k + \omega_k \cdot dt \\
\omega_{k+1} &= \omega_k - \frac{g}{L}\sin(\theta_k) \cdot dt
\end{aligned}
$$

Measurement: Observe angle only (with noise)
$$z_k = \theta_k + v_k$$

In [ ]:
# Simulate pendulum
def simulate_pendulum(dt, n_steps, process_noise_std, obs_noise_std, 
                       theta0=np.pi/4, omega0=0, g=9.81, L=1.0, seed=42):
    """Simulate nonlinear pendulum."""
    rng = np.random.default_rng(seed)
    
    f, F_jac = pendulum_dynamics(dt, g, L)
    h, H_jac = angle_observation_model()
    
    Q = np.eye(2) * process_noise_std**2
    R = np.array([[obs_noise_std**2]])
    
    true_states = []
    observations = []
    x = np.array([theta0, omega0])
    
    for _ in range(n_steps):
        # True dynamics with noise
        w = rng.multivariate_normal(np.zeros(2), Q)
        x = f(x, None) + w
        true_states.append(x.copy())
        
        # Noisy measurement
        v = rng.normal(0, obs_noise_std)
        z = h(x) + v
        observations.append(z[0])
    
    times = np.arange(n_steps) * dt
    return times, np.array(true_states), np.array(observations), f, h, F_jac, H_jac, Q, R

# Generate data
dt = 0.05
n_steps = 200
process_noise = 0.01
obs_noise = 0.1
g, L = 9.81, 1.0

times, true_states, observations, f, h, F_jac, H_jac, Q, R = simulate_pendulum(
    dt, n_steps, process_noise, obs_noise, theta0=np.pi/4, omega0=0, g=g, L=L, seed=42
)

print(f"Simulated pendulum: {n_steps} steps, dt={dt}s")
print(f"Period ≈ {2*np.pi*np.sqrt(L/g):.2f}s, Observed period ≈ {n_steps*dt:.1f}s")

In [ ]:
# Visualize ground truth
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(times, true_states[:, 0], 'b-', linewidth=2, label='True Angle')
axes[0].scatter(times, observations, c='red', s=10, alpha=0.5, label='Noisy Measurements')
axes[0].set_ylabel('Angle θ (rad)', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Pendulum Simulation', fontsize=13)

axes[1].plot(times, true_states[:, 1], 'g-', linewidth=2, label='True Angular Velocity')
axes[1].set_xlabel('Time (s)', fontsize=12)
axes[1].set_ylabel('ω (rad/s)', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'pendulum_ground_truth.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n📊 Nonlinear oscillatory motion with sin(θ) term")

## 5. Run Extended Kalman Filter

In [ ]:
# EKF
ekf = ExtendedKalmanFilter(f, h, F_jac, H_jac, Q, R)
ekf.initialize(x0=np.array([0, 0]), P0=np.eye(2) * 0.5)

ekf_estimates = []
ekf_covariances = []
ekf_time_start = time.time()

for z in observations:
    ekf.predict()
    ekf.update(np.array([z]))
    x_est, P_est = ekf.get_state()
    ekf_estimates.append(x_est)
    ekf_covariances.append(P_est)

ekf_time = time.time() - ekf_time_start
ekf_estimates = np.array(ekf_estimates)
ekf_covariances = np.array(ekf_covariances)

print(f"✓ EKF complete in {ekf_time:.4f}s")

## 6. Run Particle Filter

Test with different particle counts.

In [ ]:
# Particle Filter with varying N
particle_counts = [50, 100, 500, 1000]
pf_results = []

for N in particle_counts:
    f_stochastic = create_process_noise_wrapper(f, Q)
    likelihood = gaussian_likelihood(R)
    
    pf = ParticleFilter(f_stochastic, h, likelihood, N)
    pf.initialize(
        mean=np.array([0, 0]),
        cov=np.eye(2) * 0.5,
        rng=np.random.default_rng(42)
    )
    
    estimates = []
    eff_particles = []
    pf_time_start = time.time()
    
    for z in observations:
        pf.predict()
        pf.update(np.array([z]))
        
        # Compute effective particle count before resampling
        weights = pf.weights
        n_eff = 1.0 / np.sum(weights**2)
        eff_particles.append(n_eff)
        
        pf.resample()
        
        x_est, _ = pf.get_state_estimate()
        estimates.append(x_est)
    
    pf_time = time.time() - pf_time_start
    estimates = np.array(estimates)
    
    # Compute RMSE
    rmse_angle = np.sqrt(np.mean((true_states[:, 0] - estimates[:, 0])**2))
    rmse_omega = np.sqrt(np.mean((true_states[:, 1] - estimates[:, 1])**2))
    mean_n_eff = np.mean(eff_particles)
    
    pf_results.append({
        "N": N,
        "RMSE_angle": rmse_angle,
        "RMSE_omega": rmse_omega,
        "Time": pf_time,
        "Avg N_eff": mean_n_eff,
        "estimates": estimates,
    })
    
    print(f"✓ PF (N={N:4d}): RMSE_angle={rmse_angle:.4f}, time={pf_time:.4f}s, avg N_eff={mean_n_eff:.1f}")

# EKF RMSE
ekf_rmse_angle = np.sqrt(np.mean((true_states[:, 0] - ekf_estimates[:, 0])**2))
ekf_rmse_omega = np.sqrt(np.mean((true_states[:, 1] - ekf_estimates[:, 1])**2))

print(f"\n✓ EKF: RMSE_angle={ekf_rmse_angle:.4f}, time={ekf_time:.4f}s")

## 7. Comparison: EKF vs PF

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Angle tracking
ax = axes[0, 0]
ax.plot(times, true_states[:, 0], 'k-', linewidth=2, label='True', zorder=3)
ax.plot(times, ekf_estimates[:, 0], 'b-', linewidth=1.5, label='EKF', alpha=0.8)
ax.plot(times, pf_results[-1]['estimates'][:, 0], 'r--', linewidth=1.5, label=f'PF (N={pf_results[-1]["N"]})', alpha=0.8)
ax.set_ylabel('Angle θ (rad)', fontsize=11)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_title('Angle Tracking', fontsize=12)

# Angular velocity
ax = axes[0, 1]
ax.plot(times, true_states[:, 1], 'k-', linewidth=2, label='True', zorder=3)
ax.plot(times, ekf_estimates[:, 1], 'b-', linewidth=1.5, label='EKF', alpha=0.8)
ax.plot(times, pf_results[-1]['estimates'][:, 1], 'r--', linewidth=1.5, label=f'PF (N={pf_results[-1]["N"]})', alpha=0.8)
ax.set_ylabel('ω (rad/s)', fontsize=11)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_title('Angular Velocity Estimation', fontsize=12)

# RMSE vs N
ax = axes[1, 0]
pf_N = [r['N'] for r in pf_results]
pf_rmse = [r['RMSE_angle'] for r in pf_results]
ax.semilogx(pf_N, pf_rmse, 'ro-', linewidth=2, markersize=8, label='PF')
ax.axhline(ekf_rmse_angle, color='blue', linestyle='--', linewidth=2, label='EKF')
ax.set_xlabel('Number of Particles', fontsize=11)
ax.set_ylabel('RMSE (angle)', fontsize=11)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which='both')
ax.set_title('Accuracy vs Particle Count', fontsize=12)

# Time vs N
ax = axes[1, 1]
pf_times = [r['Time'] for r in pf_results]
ax.loglog(pf_N, pf_times, 'ro-', linewidth=2, markersize=8, label='PF')
ax.axhline(ekf_time, color='blue', linestyle='--', linewidth=2, label='EKF')
ax.set_xlabel('Number of Particles', fontsize=11)
ax.set_ylabel('Runtime (s)', fontsize=11)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which='both')
ax.set_title('Computational Cost', fontsize=12)

plt.tight_layout()
plt.savefig(output_dir / 'ekf_vs_pf_comparison.png', dpi=120, bbox_inches='tight')
plt.show()

## 8. Performance Summary Table

In [ ]:
import pandas as pd

# Build comparison table
results_table = []
results_table.append({
    "Method": "EKF",
    "N_particles": "-",
    "RMSE_angle": ekf_rmse_angle,
    "RMSE_omega": ekf_rmse_omega,
    "Time (s)": ekf_time,
    "N_eff": "-",
})

for res in pf_results:
    results_table.append({
        "Method": f"PF",
        "N_particles": res['N'],
        "RMSE_angle": res['RMSE_angle'],
        "RMSE_omega": res['RMSE_omega'],
        "Time (s)": res['Time'],
        "N_eff": res['Avg N_eff'],
    })

df_comparison = pd.DataFrame(results_table)

print("\n" + "="*80)
print("EKF vs PARTICLE FILTER: Performance Comparison")
print("="*80)
print(df_comparison.to_string(index=False))
print("="*80)

df_comparison.to_csv(output_dir / 'ekf_vs_pf_metrics.csv', index=False)

print("\n💡 Key Observations:")
print(f"   - EKF is {pf_results[-1]['Time']/ekf_time:.1f}x faster than PF (N=1000)")
print(f"   - PF (N=1000) has {(ekf_rmse_angle - pf_results[-1]['RMSE_angle'])/ekf_rmse_angle*100:.1f}% lower RMSE")
print(f"   - For this mildly nonlinear problem, EKF is sufficient!")

## 9. Diagnostic: Effective Particle Count

**Particle degeneracy**: After many updates, most weight concentrates on few particles.

**Metric**: Effective particle count
$$N_{eff} = \frac{1}{\sum_i (w^{(i)})^2}$$

Ideally $N_{eff} \approx N$. If $N_{eff} \ll N$, resample!

In [ ]:
# Analyze effective particles over time for N=500
N_test = 500
f_stochastic = create_process_noise_wrapper(f, Q)
likelihood = gaussian_likelihood(R)

pf_diag = ParticleFilter(f_stochastic, h, likelihood, N_test)
pf_diag.initialize(np.array([0, 0]), np.eye(2) * 0.5, rng=np.random.default_rng(42))

n_eff_history = []

for z in observations:
    pf_diag.predict()
    pf_diag.update(np.array([z]))
    
    # Compute N_eff BEFORE resampling
    weights = pf_diag.weights
    n_eff = 1.0 / np.sum(weights**2)
    n_eff_history.append(n_eff)
    
    pf_diag.resample()

n_eff_history = np.array(n_eff_history)

plt.figure(figsize=(12, 5))
plt.plot(times, n_eff_history, 'b-', linewidth=2, label='N_eff')
plt.axhline(N_test, color='green', linestyle='--', linewidth=2, label=f'N={N_test}')
plt.axhline(N_test * 0.5, color='red', linestyle=':', linewidth=2, label='50% threshold')
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Effective Particle Count', fontsize=12)
plt.title(f'Particle Degeneracy (N={N_test})', fontsize=13)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(output_dir / 'particle_degeneracy.png', dpi=120, bbox_inches='tight')
plt.show()

print(f"\n📊 Average N_eff: {np.mean(n_eff_history):.1f} / {N_test} ({np.mean(n_eff_history)/N_test*100:.1f}%)")
print(f"   Min N_eff: {np.min(n_eff_history):.1f}")
print("\n💡 Good N_eff → particles represent distribution well")
print("   Low N_eff → need resampling (already done automatically)")

## 10. Experiment: Highly Nonlinear Regime

**Question:** When does EKF fail?

Test with large initial angle (θ₀ = 3π/4) where linearization is poor.

In [ ]:
# Highly nonlinear regime
times_nl, true_states_nl, observations_nl, f_nl, h_nl, F_jac_nl, H_jac_nl, Q_nl, R_nl = simulate_pendulum(
    dt, n_steps, process_noise, obs_noise, theta0=3*np.pi/4, omega0=0, g=g, L=L, seed=43
)

# EKF
ekf_nl = ExtendedKalmanFilter(f_nl, h_nl, F_jac_nl, H_jac_nl, Q_nl, R_nl)
ekf_nl.initialize(x0=np.array([np.pi/2, 0]), P0=np.eye(2) * 1.0)  # Rough guess

ekf_nl_estimates = []
for z in observations_nl:
    ekf_nl.predict()
    ekf_nl.update(np.array([z]))
    x_est, _ = ekf_nl.get_state()
    ekf_nl_estimates.append(x_est)

ekf_nl_estimates = np.array(ekf_nl_estimates)

# PF
f_stoch_nl = create_process_noise_wrapper(f_nl, Q_nl)
likelihood_nl = gaussian_likelihood(R_nl)
pf_nl = ParticleFilter(f_stoch_nl, h_nl, likelihood_nl, N_particles=1000)
pf_nl.initialize(np.array([np.pi/2, 0]), np.eye(2) * 1.0, rng=np.random.default_rng(43))

pf_nl_estimates = []
for z in observations_nl:
    pf_nl.predict()
    pf_nl.update(np.array([z]))
    pf_nl.resample()
    x_est, _ = pf_nl.get_state_estimate()
    pf_nl_estimates.append(x_est)

pf_nl_estimates = np.array(pf_nl_estimates)

# Plot
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(times_nl, true_states_nl[:, 0], 'k-', linewidth=2, label='True')
axes[0].plot(times_nl, ekf_nl_estimates[:, 0], 'b-', linewidth=1.5, label='EKF', alpha=0.8)
axes[0].plot(times_nl, pf_nl_estimates[:, 0], 'r--', linewidth=1.5, label='PF (N=1000)', alpha=0.8)
axes[0].set_ylabel('Angle θ (rad)', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Highly Nonlinear Regime (θ₀ = 3π/4)', fontsize=13)

axes[1].plot(times_nl, np.abs(true_states_nl[:, 0] - ekf_nl_estimates[:, 0]), 'b-', linewidth=2, label='EKF Error')
axes[1].plot(times_nl, np.abs(true_states_nl[:, 0] - pf_nl_estimates[:, 0]), 'r-', linewidth=2, label='PF Error')
axes[1].set_xlabel('Time (s)', fontsize=12)
axes[1].set_ylabel('Absolute Error (rad)', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].set_title('Tracking Errors', fontsize=13)

plt.tight_layout()
plt.savefig(output_dir / 'highly_nonlinear_comparison.png', dpi=120, bbox_inches='tight')
plt.show()

rmse_ekf_nl = np.sqrt(np.mean((true_states_nl[:, 0] - ekf_nl_estimates[:, 0])**2))
rmse_pf_nl = np.sqrt(np.mean((true_states_nl[:, 0] - pf_nl_estimates[:, 0])**2))

print(f"\n⚠️ Highly nonlinear regime:")
print(f"   EKF RMSE: {rmse_ekf_nl:.4f} rad")
print(f"   PF RMSE:  {rmse_pf_nl:.4f} rad")
print(f"\n   PF is {(rmse_ekf_nl - rmse_pf_nl)/rmse_ekf_nl*100:.1f}% more accurate!")
print(f"   EKF struggles more when nonlinearity is strong.")

## 11. Key Takeaways

✅ **EKF**: Fast, works for mildly nonlinear systems, can diverge if linearization fails  
✅ **PF**: Handles arbitrary nonlinearity, computationally expensive (O(N))  
✅ **Trade-off**: Accuracy vs computational cost  
✅ **Effective particle count** diagnoses PF health  
✅ **When EKF fails**: Highly nonlinear regions, poor initialization  
✅ **Practical advice**: Start with EKF, switch to PF if needed  

---

## 12. Exercises

### Exercise 1: Tune Particle Count

**Task:** Find the minimum N where PF matches EKF accuracy (within 5%).

In [ ]:
# Your code here

### Exercise 2: No Resampling

**Task:** Run PF WITHOUT resampling. What happens to N_eff? Does accuracy degrade?

In [ ]:
# Your code here

### Exercise 3: Nonlinear Measurement

**Task:** Modify system to have nonlinear measurement: $z = \sin(\theta) + v$. Does EKF still work?

In [ ]:
# Your code here

---

## 13. Solutions

### Solution 1: Minimum N

In [ ]:
# Solution
target_rmse = ekf_rmse_angle * 1.05  # Within 5%

for res in pf_results:
    if res['RMSE_angle'] <= target_rmse:
        print(f"✓ Minimum N = {res['N']} to match EKF (within 5%)")
        print(f"   PF RMSE: {res['RMSE_angle']:.4f}, Target: {target_rmse:.4f}")
        break

### Solution 2: No Resampling

In [ ]:
# Solution: Run PF without resampling
pf_no_resamp = ParticleFilter(f_stochastic, h, likelihood, 500)
pf_no_resamp.initialize(np.array([0, 0]), np.eye(2) * 0.5, rng=np.random.default_rng(42))

estimates_no_resamp = []
n_eff_no_resamp = []

for z in observations:
    pf_no_resamp.predict()
    pf_no_resamp.update(np.array([z]))
    # NO RESAMPLING!
    
    weights = pf_no_resamp.weights
    n_eff_no_resamp.append(1.0 / np.sum(weights**2))
    
    x_est, _ = pf_no_resamp.get_state_estimate()
    estimates_no_resamp.append(x_est)

estimates_no_resamp = np.array(estimates_no_resamp)

plt.figure(figsize=(12, 5))
plt.plot(times, n_eff_no_resamp, 'r-', linewidth=2, label='N_eff (no resampling)')
plt.axhline(500, color='green', linestyle='--', label='N=500')
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Effective Particles', fontsize=12)
plt.title('Particle Degeneracy WITHOUT Resampling', fontsize=13)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(output_dir / 'ex2_no_resampling.png', dpi=120, bbox_inches='tight')
plt.show()

rmse_no_resamp = np.sqrt(np.mean((true_states[:, 0] - estimates_no_resamp[:, 0])**2))
print(f"\n⚠️ Without resampling:")
print(f"   Final N_eff: {n_eff_no_resamp[-1]:.1f} (started at 500)")
print(f"   RMSE: {rmse_no_resamp:.4f} (vs {pf_results[1]['RMSE_angle']:.4f} with resampling)")
print("\n   Degeneracy is severe! Resampling is essential.")

### Solution 3: Nonlinear Measurement

In [ ]:
# Solution: Nonlinear measurement h(x) = sin(θ) + v
def h_nonlinear(x):
    return np.array([np.sin(x[0])])

def H_nonlinear_jac(x):
    return np.array([[np.cos(x[0]), 0]])

# Simulate with nonlinear measurement
rng_nl = np.random.default_rng(44)
true_states_nlm = []
observations_nlm = []
x = np.array([np.pi/4, 0])

for _ in range(n_steps):
    w = rng_nl.multivariate_normal(np.zeros(2), Q)
    x = f(x, None) + w
    true_states_nlm.append(x.copy())
    
    v = rng_nl.normal(0, obs_noise)
    z = h_nonlinear(x) + v
    observations_nlm.append(z[0])

true_states_nlm = np.array(true_states_nlm)
observations_nlm = np.array(observations_nlm)

# EKF with nonlinear measurement
ekf_nlm = ExtendedKalmanFilter(f, h_nonlinear, F_jac, H_nonlinear_jac, Q, R)
ekf_nlm.initialize(np.array([0, 0]), np.eye(2))

ekf_nlm_estimates = []
for z in observations_nlm:
    ekf_nlm.predict()
    ekf_nlm.update(np.array([z]))
    x_est, _ = ekf_nlm.get_state()
    ekf_nlm_estimates.append(x_est)

ekf_nlm_estimates = np.array(ekf_nlm_estimates)

rmse_nlm = np.sqrt(np.mean((true_states_nlm[:, 0] - ekf_nlm_estimates[:, 0])**2))

print(f"\n✓ Nonlinear measurement h(x) = sin(θ):")
print(f"   EKF RMSE: {rmse_nlm:.4f} rad")
print(f"   vs linear measurement RMSE: {ekf_rmse_angle:.4f} rad")
print("\n💡 EKF still works, but accuracy may degrade with strong nonlinearity.")
print("   For very nonlinear h, consider PF or Unscented KF!")

---

## Summary Report

In [ ]:
summary = f"""
# Notebook 03: Nonlinear Estimation - Summary

**Date:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

## Key Results

1. **EKF vs PF (Pendulum):**
   - EKF: RMSE = {ekf_rmse_angle:.4f} rad, Time = {ekf_time:.4f}s
   - PF (N=1000): RMSE = {pf_results[-1]['RMSE_angle']:.4f} rad, Time = {pf_results[-1]['Time']:.4f}s
   - Speedup: EKF is {pf_results[-1]['Time']/ekf_time:.1f}x faster

2. **Particle Count Trade-off:**
   - N=50: Poor accuracy, fast
   - N=1000: Best accuracy, slowest
   - Sweet spot: ~200-500 particles for this problem

3. **Highly Nonlinear Regime:**
   - Large initial angle (θ₀=3π/4)
   - PF outperforms EKF by {(rmse_ekf_nl - rmse_pf_nl)/rmse_ekf_nl*100:.1f}%

## Outputs
   - pendulum_ground_truth.png
   - ekf_vs_pf_comparison.png
   - particle_degeneracy.png
   - highly_nonlinear_comparison.png
   - ekf_vs_pf_metrics.csv

## Next Steps

→ Notebook 04: Time series forecasting with proper backtesting
"""

with open(output_dir / 'summary.md', 'w') as f:
    f.write(summary)

print("\n" + "="*60)
print("✓ Notebook 03 Complete!")
print("="*60)
print(f"Outputs saved to: {output_dir}")